In [1]:
#对方法进行封装，形成模板

In [13]:
import requests
import re
from lxml import etree
from queue import Queue

In [ ]:
#从01字符串中解析出各标识符的True/False值
def getIdenti(identi):
    #解析方法

In [8]:
#待爬取网页类型

#False为静态，True为动态
webIsDynamic = False 

#False为一层，True为两层
webDepth = False 

#False在动态网页下：为指定pagenum范围；静态网页下为：截取当前url获得
#True为通过爬取当前网页才能获得
pagenumByCra = False

In [12]:
#初始url
ini_url = ''


#若pagenumByCra为false，需要输入pagenum范围或xpath
pagenum_false = ''
#若pagenumByCra为true,输入爬取所需的xpath
pagenum_true = ''

#爬取第一层需要的参数：获取二次爬取url列表所需的xpath
page_para1 = ''
#爬取第二层需要的参数1
page_para21 = ''
page_para22 = ''

In [4]:

def getPage(url,webIsDynamic,headers):
    try:
        if webIsDynamic:
            #对于动态网页的请求方法
            r = requests.get(url,headers=headers)
        else:
            #静态网页请求方法
            r = requests.get(url)
        
        html = r.text
        return html
    except ValueError:
        print("输入无效")
        
    

In [5]:
def parsePagelxml(html):
    #静态网页请求后需要用lxml解析
    s = etree.HTML(html)
    return s

In [6]:
#用截取方式获得想要的字符串
def getContentCut(regex,ini_str):
    matchObj = re.match(r''+regex,ini_str)
    Content = matchObj.group(1)
    return Content

In [7]:
#用爬取的方式获得想要的字符串
def getContentCra(xpath,url):
    html = getPage(url,webIsDynamic,headers)
    s = parsePagelxml(html)
    content = s.xpath(xpath)
    return content

In [ ]:
#单层爬取的参数获取
def getPagenumPara(pagenumByCra):
    if pagenumByCra:
        param = pagenum_true
    else:
        parem = pagenum_false
    
    return param

In [ ]:
#双层爬取的参数获取
def getDouPagenumPara():
    #第一层的参数
    para1 = page_para1
    #第二层的参数
    para21 = page_para21
    para22 = page_para22
    
    
    return para1,para21,para22

In [ ]:
def getPagenum(url,webIsDynamic,pagenumByCra):
    #通过某个url获得pagenum或许是截或许是爬，或许是直接指定
    param = getPagenumPara(pagenumByCra)
    
    if pagenumByCra:
        #pagenum需要通过爬取网页得到
        pagenum_str = getContentCra(param,ini_url)
        pagenum = int(pagenum_str)
        
    else:
        if webIsDynamic:
            pagenum = int(param)
        else:
            pagenum_str = getContentCut(param,ini_url)
            pagenum = int(pagenum_str)
            
    return pagenum

In [ ]:
#获得pagenum和url_model的各部分
#可以再将两层拆开为两个封装好的方法，进行调用
def getUrlStr(ini_url,headers,webDepth,webIsDynamic,pagenumByCra):
    url_model = []
    pagenum = []
    
    if webDepth:
        #两层应该进行的操作
        para1,para21,para22 = getDouPagenumPara()
        #首先获取下一次爬取的队列
        html = getPage(ini_url,webIsDynamic,headers)
        s = parsePagelxml(html)
        url = getContentCra(para1,ini_url)
        queueYear = Queue()
        for i in range(1,len(url)):
            queueYear.put(url[i])
        
        #第二次爬取的操作
        queueLast = Queue()
        while not queueYear.empty():
            nexturl = queueYear.get()
            html2 = getPage(nexturl,webIsDynamic,headers)
            s2 = parsePagelxml(html2)
            pagenumUrl = s2.xpath(para21)
            queueLast.put(next_content[0])
            
        while not queueLast.empty():
            lasturl = queueLast.get()
            matchObj1 = re.match(r''+para22,lasturl,re.M | re.U)
            
            url_mo = matchObj1.group(1)
            url_model.append(url_mo)
            
            pagenum_here = matchObj1.group(2)
            pagenum.append(pagenum_here)
            #爬两层的网页中，需要model和pagenum一一对应
            
       
        
    else:
        #一层应该进行的操作
        pagenum_here = getPagenum(ini_url,webIsDynamic,pagenumByCra)
        pagenum.append(pagenum_here)
        
        
        
        